# third_party - draw

## Notebook运行提示
- 代码已拆分为多个小单元, 按顺序运行即可在每一步观察输出与中间变量。
- 涉及 `Path(__file__)` 或相对路径的脚本会自动注入 `__file__` 解析逻辑, Notebook 环境下也能引用原项目资源。
- 可在每个单元下追加说明或参数试验记录, 以跟踪核心算法和数据处理步骤。


In [ ]:
# Notebook路径自适应处理
import pathlib as _nb_pathlib
def _nb_resolve_file_path():
    if '__file__' not in globals():
        _cwd = _nb_pathlib.Path.cwd().resolve()
        for _candidate in (_cwd, *_cwd.parents):
            _potential = _candidate / '09-practical-projects/05_Kaggle竞赛项目/04-RSNA-2024-Lumbar-Spine/src/third_party/my_lib/draw.py'
            if _potential.exists():
                globals()['__file__'] = str(_potential)
                return
        globals()['__file__'] = str((_cwd / '09-practical-projects/05_Kaggle竞赛项目/04-RSNA-2024-Lumbar-Spine/src/third_party/my_lib/draw.py').resolve())
_nb_resolve_file_path()
del _nb_pathlib


In [ ]:
import os
#qt bug ???
os.environ['QT_XKB_CONFIG_ROOT']='/usr/share/X11/xkb/'

import cv2
import numpy as np
import matplotlib.cm

In [ ]:


# draw -----------------------------------
def image_show(name, image, type='bgr', resize=1):
    if type == 'rgb': image = np.ascontiguousarray(image[:,:,::-1])
    H,W = image.shape[0:2]
    
    cv2.namedWindow(name, cv2.WINDOW_GUI_NORMAL)  #WINDOW_NORMAL
    #cv2.namedWindow(name, cv2.WINDOW_GUI_EXPANDED)  #WINDOW_GUI_EXPANDED
    image =cv2.resize(image,(round(resize*W), round(resize*H) ))
    cv2.imshow(name, image) #.astype(np.uint8))
    cv2.resizeWindow(name, round(resize*W), round(resize*H))

In [ ]:


def image_show_norm(name, image, min=None, max=None, type='bgr', resize=1):
    if max is None: max=image.max()
    if min is None: min=image.min()
    if type == 'rgb': image = np.ascontiguousarray(image[:,:,::-1])
 
    H,W = image.shape[0:2]
    rW, rH = round(resize * W), round(resize * H)
    rimage = cv2.resize(image,(rW,rH)) #image #
    cv2.namedWindow(name, cv2.WINDOW_GUI_NORMAL)  #WINDOW_NORMAL WINDOW_GUI_NORMAL
    cv2.resizeWindow(name, rW, rH)
    cv2.imshow(name, (np.clip((rimage-min)/(max-min+0.0001),0,1)*255).astype(np.uint8))

In [ ]:


def draw_shadow_text(img, text, pt,  fontScale, color, thickness, color1=None, thickness1=None):
    if color1 is None: color1=(0,0,0)
    if thickness1 is None: thickness1 = thickness+2

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, text, pt, font, fontScale, color1, thickness1, cv2.LINE_AA)
    if color1!=color:
        cv2.putText(img, text, pt, font, fontScale, color,  thickness,  cv2.LINE_AA)

In [ ]:

def to_color_image(image, max=None):
    if max is None: max=image.max()

    image = (image/max*255).astype(np.uint8)
    image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
    return image

In [ ]:



##http://stackoverflow.com/questions/26690932/opencv-rectangle-with-dotted-or-dashed-lines
def draw_dotted_line(image, pt1, pt2, color, thickness=1, gap=20):

    dist =((pt1[0]-pt2[0])**2+(pt1[1]-pt2[1])**2)**.5
    pts= []
    for i in  np.arange(0,dist,gap):
        r=i/dist
        x=int((pt1[0]*(1-r)+pt2[0]*r)+.5)
        y=int((pt1[1]*(1-r)+pt2[1]*r)+.5)
        p = (x,y)
        pts.append(p)

    if gap==1:
        for p in pts:
            cv2.circle(image,p,thickness,color,-1,cv2.LINE_AA)
    else:
        def pairwise(iterable):
            "s -> (s0, s1), (s2, s3), (s4, s5), ..."
            a = iter(iterable)
            return zip(a, a)

        for p, q in pairwise(pts):
            cv2.line(image,p, q, color,thickness,cv2.LINE_AA)

In [ ]:


def draw_dotted_poly(image, pts, color, thickness=1, gap=20, is_close=True ):
    s=pts[0]
    e=pts[0]
    if is_close: pts.append(pts.pop(0))
    for p in pts:
        s=e
        e=p
        draw_dotted_line(image,s,e,color,thickness,gap)

In [ ]:


def draw_dotted_rect(image, pt1, pt2, color, thickness=1, gap=3):
    pts = [pt1,(pt2[0],pt1[1]),pt2,(pt1[0],pt2[1])]
    draw_dotted_poly(image, pts, color, thickness, gap)

In [ ]:

def draw_screen_rect(image, pt1, pt2, color, alpha=0.5):
    x1, y1 = pt1
    x2, y2 = pt2
    image[y1:y2,x1:x2,:] = (1-alpha)*image[y1:y2,x1:x2,:] + (alpha)*np.array(color, np.uint8)

In [ ]:


def draw_screen_circle(image, pt, radius, color, alpha=0.5):
    s = 2*radius+1
    m = np.zeros((s,s), np.uint8)
    cv2.circle(m,(radius,radius),radius,255,-1)

    height,width = image.shape[:2]
    y,x = np.where(m!=0)
    y += pt[1]-radius
    x += pt[0]-radius

    m = (x>=0) & (y>=0) & (x<width) & (y<height)
    y = y[m]
    x = x[m]
    image[y,x] = (1-alpha)*image[y,x] + (alpha)*np.array(color, np.uint8)

In [ ]:

# def draw_mask(image, mask, color=(255,255,255), α=1,  β=0.25, λ=0., threshold=32 ):
#     # image * α + mask * β + λ
#
#     if threshold is None:
#         mask = mask/255
#     else:
#         mask = clean_mask(mask,threshold,1)
#
#     mask  = np.dstack((color[0]*mask,color[1]*mask,color[2]*mask)).astype(np.uint8)
#     image[...] = cv2.addWeighted(image, α, mask, β, λ)
#


# def draw_contour(image, mask, color=(0,255,0), thickness=1, threshold=127):
#     ret, thresh = cv2.threshold(mask,threshold,255,0)
#     ret = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     hierarchy = ret[0]
#     contours  = ret[1]
#     #image[...]=image
#     cv2.drawContours(image, contours, -1, color, thickness, cv2.LINE_AA)
#     ## drawContours(image, contours, contourIdx, color, thickness=None, lineType=None, hierarchy=None, maxLevel=None, offset=None): # real signature unknown; restored from __doc__
#
#


def to_color(s, color=None):

    if type(color) in [str] or color is None:
        #https://matplotlib.org/xkcd/examples/color/colormaps_reference.html

        if color is None: color='cool'
        color = matplotlib.get_cmap(color)(s)
        b = int(255*color[2])
        g = int(255*color[1])
        r = int(255*color[0])

    elif type(color) in [list,tuple]:
        b = int(s*color[0])
        g = int(s*color[1])
        r = int(s*color[2])

    return (b,g,r)

In [ ]:



# main #################################################################
if __name__ == '__main__':
    print( '%s: calling main function ... ' % os.path.basename(__file__))


    image = np.zeros((50,50,3), np.uint8)
    cv2.rectangle(image, (0,0),(49,49), (0,0,255),1) #inclusive

    image[8,8]=[255,255,255]

    image_show('image',image,10)
    cv2.waitKey(0)


    print('\nsucess!')